In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [5]:
url <- "https://raw.githubusercontent.com/yvinc/DSCI100-Group-Project/main/oasis_longitudinal.csv?token=GHSAT0AAAAAACDJOPLWP2P6VGJHJQLPARFQZECGDRA"
read_csv(url)

ERROR: Error in open.connection(structure(6L, class = c("curl", "connection"), conn_id = <pointer: 0x235>), : HTTP error 404.
